In [2]:
import pandas as pd
import pandas as pd
from utils import snowflake_login, descargar_segmento, get_credentials
from datetime import datetime
import os

In [9]:
repositorio = 'Proyectos_Streamlit\\Pricing'                 #Definimos repositorio para obtener path base
path_base = os.getcwd()[:os.getcwd().find(repositorio)]
path_base

'c:\\Users\\marcos.larran\\Desktop\\Github\\'

In [6]:
#Conectamos a snowflake
credentials_snowflake = get_credentials("snow")
user, cursor, snow = snowflake_login(
                                    user = credentials_snowflake['USER'],
                                    password = credentials_snowflake['PASS'],
                                    account = credentials_snowflake['ACCOUNT']
                                    )

Intento 1


2026-02-11 16:33:54.000 
  command:

    streamlit run c:\Users\marcos.larran\AppData\Local\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


Correct Password - connected to SNOWFLAKE


In [9]:
ref = descargar_segmento(cursor, 'CANASTAS', conds=[" WHERE CANASTA = 'referente'"])
ref

,GEOG_LOCL_ID,ITEM,CANASTA
0,2,1000049025,referente
1,24,1000049025,referente
2,32,1000049025,referente
3,49,1000049025,referente
4,61,1000049025,referente
...,...,...,...
41449,91,1000436442,referente
41450,73,1000436442,referente
41451,134,1000436442,referente
41452,13,1000436442,referente


In [18]:
can = descargar_segmento(cursor, 'CANASTAS', conds=[""])#.astype('str')
can

,GEOG_LOCL_ID,ITEM,CANASTA
0,2,1000049025,referente
1,24,1000049025,referente
2,32,1000049025,referente
3,49,1000049025,referente
4,61,1000049025,referente
...,...,...,...
4056001,91,1000051862,surtido
4056002,73,1000051862,surtido
4056003,134,1000051862,surtido
4056004,13,1000051862,surtido


In [19]:
can.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4056006 entries, 0 to 4056005
Data columns (total 3 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   GEOG_LOCL_ID  int16 
 1   ITEM          int32 
 2   CANASTA       object
dtypes: int16(1), int32(1), object(1)
memory usage: 54.2+ MB


In [21]:
can['ITEM'] = can['ITEM'].astype('str')

In [23]:
tuple(can.iloc[:200]['ITEM'].unique())

('1000049025', '1000049028', '1000048276')

In [25]:
query = 'DELETE FROM SANDBOX_PLUS.DWH.CANASTAS2 WHERE ITEM IN {items}'
cursor.execute(query.format(items=tuple(can.iloc[:200]['ITEM'].unique())))

In [3]:
uploaded_file = pd.read_excel('../../cargaInicial.xlsx')
uploaded_file

,ORIN,MONTEVIDEO,CANELONES,SAN JOSE,CERRO LARGO,DURAZNO,TACUAREMBÓ,RIVERA,ARTIGAS,TREINTA Y TRES,LAVALLEJA,MALDONADO,FLORES,ROCHA,FLORIDA,SALTO,SORIANO,RIO NEGRO,COLONIA,PAYSANDU
0,1000049025,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0
1,1000049028,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0
2,1000048880,1,1,0,1,1,0,1,1,1,0,1,1,1,0,1,0,0,1,0
3,1000048984,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1
4,1000049023,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565,1000037695,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
566,1000037689,1,1,1,1,1,1,0,0,1,1,1,0,0,0,0,1,0,0,0
567,1000037633,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1
568,1000037636,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1


In [69]:
#Armamos archivos con referentes
referentes = uploaded_file.melt(id_vars='ORIN', value_vars=uploaded_file.columns[1:], var_name='GEOG_DPTO_DESC',
                                value_name='REFERENTE').rename(columns={'ORIN':'ITEM'})
referentes['aux'] = 1
referentes = referentes[referentes['REFERENTE']==1].drop(columns='REFERENTE')
referentes

,ITEM,GEOG_DPTO_DESC,aux
2,1000048880,MONTEVIDEO,1
3,1000048984,MONTEVIDEO,1
6,1000048971,MONTEVIDEO,1
7,1000048967,MONTEVIDEO,1
9,1000187935,MONTEVIDEO,1
...,...,...,...
10823,1000040363,PAYSANDU,1
10824,1000164208,PAYSANDU,1
10827,1000037633,PAYSANDU,1
10828,1000037636,PAYSANDU,1


In [70]:
#Descargamos los locales y los referentes actuales
locales = descargar_segmento(cursor, 'LOCALES')
locales['aux'] = 1
locales

,GEOG_LOCL_ID,GEOG_DPTO_DESC,aux
0,2,SALTO,1
1,3,CANELONES,1
2,4,SORIANO,1
3,5,TACUAREMBÓ,1
4,191,MONTEVIDEO,1
...,...,...,...
89,102,FLORES,1
90,302,CANELONES,1
91,295,LAVALLEJA,1
92,303,MALDONADO,1


In [71]:
ref = descargar_segmento(cursor, 'CANASTAS', conds=["'referente'"])
ref['aux'] = 1
ref

,GEOG_LOCL_ID,ITEM,CANASTA,aux
0,2,1000049025,referente,1
1,24,1000049025,referente,1
2,32,1000049025,referente,1
3,49,1000049025,referente,1
4,61,1000049025,referente,1
...,...,...,...,...
41449,91,1000436442,referente,1
41450,73,1000436442,referente,1
41451,134,1000436442,referente,1
41452,13,1000436442,referente,1


In [72]:
mercado1 = ref[~ref['ITEM'].isin(referentes['ITEM'])].drop(columns='GEOG_LOCL_ID').drop_duplicates().merge(locales)[ref.columns].drop(columns='aux')
mercado1

,GEOG_LOCL_ID,ITEM,CANASTA
0,2,1000049120,referente
1,3,1000049120,referente
2,4,1000049120,referente
3,5,1000049120,referente
4,191,1000049120,referente
...,...,...,...
18043,102,1000056479,referente
18044,302,1000056479,referente
18045,295,1000056479,referente
18046,303,1000056479,referente


In [73]:
mercado1['ITEM'].value_counts()

ITEM
1000049120    94
1000049195    94
1000041930    94
1000041824    94
1000041943    94
              ..
1000460709    94
1000055868    94
1000054577    94
1000050419    94
1000056479    94
Name: count, Length: 192, dtype: int64

In [77]:
carga = referentes.merge(locales).drop(columns=['aux', 'GEOG_DPTO_DESC'])
carga['CANASTA'] = 'referente'
carga

,ITEM,GEOG_LOCL_ID,CANASTA
0,1000048880,191,referente
1,1000048880,229,referente
2,1000048880,24,referente
3,1000048880,25,referente
4,1000048880,28,referente
...,...,...,...
31213,1000037636,7,referente
31214,1000037654,18,referente
31215,1000037654,19,referente
31216,1000037654,20,referente


In [75]:
carga['ITEM'].value_counts()

ITEM
1000050434    86
1000523176    86
1000570013    86
1000050762    86
1000050800    86
              ..
1000048564     1
1000052028     1
1000461487     1
1000051908     1
1000048566     1
Name: count, Length: 570, dtype: int64

In [55]:
referentes

,ITEM,GEOG_DPTO_DESC,aux
2,1000048880,MONTEVIDEO,1
3,1000048984,MONTEVIDEO,1
6,1000048971,MONTEVIDEO,1
7,1000048967,MONTEVIDEO,1
9,1000187935,MONTEVIDEO,1
...,...,...,...
10823,1000040363,PAYSANDU,1
10824,1000164208,PAYSANDU,1
10827,1000037633,PAYSANDU,1
10828,1000037636,PAYSANDU,1


In [56]:
referentes['ITEM'].unique().size

570

In [57]:
carga

,ITEM,GEOG_LOCL_ID
0,1000048880,191
1,1000048880,229
2,1000048880,24
3,1000048880,25
4,1000048880,28
...,...,...
31213,1000037636,7
31214,1000037654,18
31215,1000037654,19
31216,1000037654,20


In [58]:
locales.drop(columns='GEOG_DPTO_DESC').merge(referentes.drop(columns='GEOG_DPTO_DESC').drop_duplicates()).drop(columns='aux')

,GEOG_LOCL_ID,ITEM
0,2,1000048880
1,2,1000048984
2,2,1000048971
3,2,1000048967
4,2,1000187935
...,...,...
53575,322,1000099231
53576,322,1000099234
53577,322,1000057461
53578,322,1000048124


In [40]:
carga

,ORIN,GEOG_LOCL_ID
0,1000048880,191
1,1000048880,229
2,1000048880,24
3,1000048880,25
4,1000048880,28
...,...,...
31213,1000037636,7
31214,1000037654,18
31215,1000037654,19
31216,1000037654,20


In [79]:
mercado2 = locales.drop(columns='GEOG_DPTO_DESC').merge(referentes.drop(columns='GEOG_DPTO_DESC').drop_duplicates()).drop(columns='aux').merge(carga, how='left', indicator=True)
mercado = pd.concat([mercado1, mercado2[mercado2['_merge']!='both'].drop(columns='_merge')], ignore_index=True)
mercado['CANASTA'] = 'mercado'
mercado

,GEOG_LOCL_ID,ITEM,CANASTA
0,2,1000049120,mercado
1,3,1000049120,mercado
2,4,1000049120,mercado
3,5,1000049120,mercado
4,191,1000049120,mercado
...,...,...,...
40405,322,1000099231,mercado
40406,322,1000099234,mercado
40407,322,1000057461,mercado
40408,322,1000048124,mercado


In [61]:
merca = descargar_segmento(cursor, 'CANASTAS', conds=["'mercado'"])
merca

,GEOG_LOCL_ID,ITEM,CANASTA
0,2,1000049023,mercado
1,24,1000049023,mercado
2,32,1000049023,mercado
3,49,1000049023,mercado
4,61,1000049023,mercado
...,...,...,...
327021,91,1000053222,mercado
327022,73,1000053222,mercado
327023,134,1000053222,mercado
327024,13,1000053222,mercado


In [67]:
merca[~merca['ITEM'].isin(mercado['ITEM'].unique())]#['ITEM'].value_counts()

,GEOG_LOCL_ID,ITEM,CANASTA
188,2,1000162728,mercado
189,24,1000162728,mercado
190,32,1000162728,mercado
191,49,1000162728,mercado
192,61,1000162728,mercado
...,...,...,...
327021,91,1000053222,mercado
327022,73,1000053222,mercado
327023,134,1000053222,mercado
327024,13,1000053222,mercado


In [80]:
print(f'asdasd {len(carga)} sadasd')

asdasd 31218 sadasd


In [68]:
carga_mercado = pd.concat([mercado, merca[~merca['ITEM'].isin(mercado['ITEM'].unique())]], ignore_index=True)
carga_mercado

,GEOG_LOCL_ID,ITEM,CANASTA
0,2,1000049120,mercado
1,3,1000049120,mercado
2,4,1000049120,mercado
3,5,1000049120,mercado
4,191,1000049120,mercado
...,...,...,...
339607,91,1000053222,mercado
339608,73,1000053222,mercado
339609,134,1000053222,mercado
339610,13,1000053222,mercado


In [22]:
locales = pd.read_csv('C:/Users/marcos.larran/Downloads/LOCALES.csv')
locales

,GEOG_LOCL_ID,CODIGO_TIENDA
0,8,132
1,9,134
2,11,140
3,12,143
4,13,144
...,...,...
240,333,450
241,334,451
242,335,452
243,336,454


In [2]:
nielsen = pd.read_csv('C:/Users/marcos.larran/Downloads/Nielsen viejo.csv')
nielsen

,STORE_ID,STORE,STORE_ADDRESS,AREA,SUB_AREA,CITY,RETAILER,BANNER,LOCL_COD,PRIORIDAD,GEOG_LOCL_ID
0,100356243,DEVOTO EXPRESS SUC. 3113 SARANDI 690 A JUNCAL ...,S R NDI 690- JUNC L 1347 NN,GRAN MONTEVIDEO B,MONTEVIDEO,NaN,DEVOTO,DEVOTO EXPRESS,101,6,21
1,100356250,DEVOTO EXPRESS SUC. 3121 CANELONES 1251 ESQ. C...,C NELONES 1251 ESQ C QUIJ NO NN,GRAN MONTEVIDEO B,MONTEVIDEO,NaN,DEVOTO,DEVOTO EXPRESS,101,7,21
2,100356180,DISCO SUC. 117 MALDONADO 1024 SUR GRANMONTEVIDEO,M LDON DO-1024-SUR NN,GRAN MONTEVIDEO B,MONTEVIDEO,BARRIO SUR,DISCO,DISCO,101,9,21
3,100356170,DISCO SUC. 103 ARENAL GDE. 1376 CORDON GRANMON...,REN L GDE-1376-CORDON NN,GRAN MONTEVIDEO B,MONTEVIDEO,CORDON,DISCO,DISCO,101,10,21
4,100356367,DEVOTO EXPRESS SUC. 3132 SAN JOSE 1085 / 1087 0,S N JOSE 1085 / 1087 NN,GRAN MONTEVIDEO B,MONTEVIDEO,NaN,DEVOTO,DEVOTO EXPRESS,101,8,21
...,...,...,...,...,...,...,...,...,...,...,...
430,100356186,DISCO SUC. 125 MCAL. SOLANO LOPEZ 1680 0,NN NN,GRAN MONTEVIDEO A,MONTEVIDEO,BUCEO,DISCO,DISCO,420,3,322
431,100356252,DEVOTO EXPRESS SUC. 3124 CNO. CARRASCO 6955 SE...,CNO C RR SCO 6955 NN,GRAN MONTEVIDEO A,MONTEVIDEO,NaN,DEVOTO,DEVOTO EXPRESS,420,2,322
432,100356159,TIENDA INGLESA SUC. 1 AV. ITALIA ESQ. BOLIVIA ...,V IT LI--ESQ BOLIVI- 5820-M LVIN NN,GRAN MONTEVIDEO A,CARRASCO,CARRASCO,TIENDA INGLESA,TIENDA INGLESA,420,4,322
433,100356160,TIENDA INGLESA SUC. 2 SCHROEDER 6436 CARRASCO ...,SCHROEDER- 6436-C RR SCO NN,GRAN MONTEVIDEO A,CARRASCO,CARRASCO,TIENDA INGLESA,TIENDA INGLESA,420,5,322


In [23]:
c_nielsen = pd.read_excel('C:/Users/marcos.larran/Downloads/correcciones Nielsen.xlsx')
print(c_nielsen.shape)
c_nielsen

(21, 2)


,CODIGO_TIENDA,STORE_ID
0,143,108282396
1,111,108282396
2,156,108282396
3,316,108282396
4,145,100356446
5,335,100356446
6,139,100356353
7,139,106558845
8,155,100356353
9,155,106558845


In [16]:
info_nielsen = pd.read_csv('C:/Users/marcos.larran/Downloads/PriceMaster_UY_grupo_tata.csv',
                           encoding='ISO-8859-1', sep='|')
info_nielsen = info_nielsen[['STORE_ID', 'STORE', 'STORE_ADDRESS', 'AREA', 'SUB_AREA',
                             'CITY', 'RETAILER', 'BANNER']].drop_duplicates()
info_nielsen

C:\Users\marcos.larran\AppData\Local\Temp\ipykernel_22396\395866159.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  info_nielsen = pd.read_csv('C:/Users/marcos.larran/Downloads/PriceMaster_UY_grupo_tata.csv',


,STORE_ID,STORE,STORE_ADDRESS,AREA,SUB_AREA,CITY,RETAILER,BANNER
0,100356162,TIENDA INGLESA SUC. 4 J.B. BLANCO 966 POCITOS ...,JB BL NCO- 966-POCITOS NN,GRAN MONTEVIDEO A,MONTEVIDEO,POCITOS,TIENDA INGLESA,TIENDA INGLESA
163,100356169,DISCO SUC. 102 AV. AGRACIADA 2986 PRADO GRANMO...,V-GR CI D- 2986-PR DO NN,GRAN MONTEVIDEO B,MONTEVIDEO,CAPURRO,DISCO,DISCO
708,100356175,DISCO SUC. 111 8 DE OCTUBRE 2681 TRES CRUCES G...,8 DE OCTUBRE- 2681-TRES CRUCES NN,GRAN MONTEVIDEO B,MONTEVIDEO,PQE BATTLE,DISCO,DISCO
798,100356176,DISCO SUC. 112 8 DE OCTUBRE 4786 UNION GRANMON...,8 DE OCTUBRE-4786-UNION NN,GRAN MONTEVIDEO B,MONTEVIDEO,MARONAS,DISCO,DISCO
1230,100356177,DISCO SUC. 113 CHUCARRO 1320 POCITOS GRANMONTE...,CHUC RRO-1320-POCITOS NN,GRAN MONTEVIDEO A,MONTEVIDEO,POCITOS,DISCO,DISCO
...,...,...,...,...,...,...,...,...
1441042,245853700,EL DORADOSUC 2510DR MORQUIO 600 ESQUINA EUSEBI...,DR MORQUIO 600 ESQUINA EUSEBIO PIREZ NN,INTERIOR CENTRO,DURAZNO,NaN,EL DORADO,EL DORADO
1445679,245853701,EL DORADOSUC 57019 DE ABRIL Y GRAL ARTIGASAMAL...,19 DE ABRIL Y GRAL ARTIGASA NN,INTERIOR ESTE,MALDONADO,NaN,EL DORADO,EL DORADO
1461395,287358763,EL DORADOSUC 2320ÿESPINILLO Y CORONILLAÿFRAY B...,ÿESPINILLO Y CORONILLAÿ NN,INTERIOR LITORAL,RIO NEGRO,NaN,EL DORADO,EL DORADO
1461452,288159883,TATASUC 420GENERAL FLORES 2928MONTEVIDEO,GENERAL FLORES 2928 NN,GRAN MONTEVIDEO B,MONTEVIDEO,NaN,TATA,TATA


In [20]:
info_nielsen['STORE_ID'].value_counts()

STORE_ID
100356162    1
100356449    1
100356445    1
100356444    1
100356443    1
            ..
100356145    1
100356141    1
100356206    1
100356205    1
305903691    1
Name: count, Length: 328, dtype: int64

In [24]:
nielsen_nuevo = c_nielsen.merge(info_nielsen).merge(locales)
print(nielsen_nuevo.shape)
nielsen_nuevo.head()

(21, 10)


,CODIGO_TIENDA,STORE_ID,STORE,STORE_ADDRESS,AREA,SUB_AREA,CITY,RETAILER,BANNER,GEOG_LOCL_ID
0,143,108282396,EL DORADO SUC. 1710 BLVR. ARTIGAS Y AVENIDA SALTO,BLVR. ARTIGAS Y AVENIDA SALTO NN,INTERIOR LITORAL,PAYSANDU,NaN,EL DORADO,EL DORADO,12
1,111,108282396,EL DORADO SUC. 1710 BLVR. ARTIGAS Y AVENIDA SALTO,BLVR. ARTIGAS Y AVENIDA SALTO NN,INTERIOR LITORAL,PAYSANDU,NaN,EL DORADO,EL DORADO,2
2,156,108282396,EL DORADO SUC. 1710 BLVR. ARTIGAS Y AVENIDA SALTO,BLVR. ARTIGAS Y AVENIDA SALTO NN,INTERIOR LITORAL,PAYSANDU,NaN,EL DORADO,EL DORADO,47
3,316,108282396,EL DORADO SUC. 1710 BLVR. ARTIGAS Y AVENIDA SALTO,BLVR. ARTIGAS Y AVENIDA SALTO NN,INTERIOR LITORAL,PAYSANDU,NaN,EL DORADO,EL DORADO,83
4,145,100356446,EL DORADO SUC. 1220 LIBERTAD 1034 SARANDI GRANDE,LIBERT D 1034 NN,INTERIOR CENTRO,FLORIDA,CENTRO,EL DORADO,EL DORADO,14


In [25]:
nielsen.columns

Index(['STORE_ID', 'STORE', 'STORE_ADDRESS', 'AREA', 'SUB_AREA', 'CITY',
       'RETAILER', 'BANNER', 'LOCL_COD', 'PRIORIDAD', 'GEOG_LOCL_ID'],
      dtype='object')

In [27]:
nielsen_nuevo = nielsen_nuevo[['STORE_ID', 'STORE', 'STORE_ADDRESS', 'AREA', 'SUB_AREA', 'CITY',
                               'RETAILER', 'BANNER', 'CODIGO_TIENDA', 'GEOG_LOCL_ID']]
nielsen_nuevo.columns = ['STORE_ID', 'STORE', 'STORE_ADDRESS', 'AREA', 'SUB_AREA', 'CITY',
                         'RETAILER', 'BANNER', 'LOCL_COD', 'GEOG_LOCL_ID']
nielsen_nuevo

,STORE_ID,STORE,STORE_ADDRESS,AREA,SUB_AREA,CITY,RETAILER,BANNER,LOCL_COD,GEOG_LOCL_ID
0,108282396,EL DORADO SUC. 1710 BLVR. ARTIGAS Y AVENIDA SALTO,BLVR. ARTIGAS Y AVENIDA SALTO NN,INTERIOR LITORAL,PAYSANDU,NaN,EL DORADO,EL DORADO,143,12
1,108282396,EL DORADO SUC. 1710 BLVR. ARTIGAS Y AVENIDA SALTO,BLVR. ARTIGAS Y AVENIDA SALTO NN,INTERIOR LITORAL,PAYSANDU,NaN,EL DORADO,EL DORADO,111,2
2,108282396,EL DORADO SUC. 1710 BLVR. ARTIGAS Y AVENIDA SALTO,BLVR. ARTIGAS Y AVENIDA SALTO NN,INTERIOR LITORAL,PAYSANDU,NaN,EL DORADO,EL DORADO,156,47
3,108282396,EL DORADO SUC. 1710 BLVR. ARTIGAS Y AVENIDA SALTO,BLVR. ARTIGAS Y AVENIDA SALTO NN,INTERIOR LITORAL,PAYSANDU,NaN,EL DORADO,EL DORADO,316,83
4,100356446,EL DORADO SUC. 1220 LIBERTAD 1034 SARANDI GRANDE,LIBERT D 1034 NN,INTERIOR CENTRO,FLORIDA,CENTRO,EL DORADO,EL DORADO,145,14
5,100356446,EL DORADO SUC. 1220 LIBERTAD 1034 SARANDI GRANDE,LIBERT D 1034 NN,INTERIOR CENTRO,FLORIDA,CENTRO,EL DORADO,EL DORADO,335,102
6,100356353,EL DORADO SUC. 1250 CARLOS BERRUTI ESQ FLORENC...,C RLOS BERRUTI ESQUIN-FLORENCIO S NCHES NN,INTERIOR NORDESTE,TACUAREMBO,NaN,EL DORADO,EL DORADO,139,10
7,106558845,EL DORADOSUC. 1260AV ANGELA B LOPEZ Y FLORENCI...,AVENIDA ANGELA B DE LOPEZ ESQUINA FLORENCIO SA...,INTERIOR NORDESTE,TACUAREMBO,CORDON,EL DORADO,EL DORADO,139,10
8,100356353,EL DORADO SUC. 1250 CARLOS BERRUTI ESQ FLORENC...,C RLOS BERRUTI ESQUIN-FLORENCIO S NCHES NN,INTERIOR NORDESTE,TACUAREMBO,NaN,EL DORADO,EL DORADO,155,45
9,106558845,EL DORADOSUC. 1260AV ANGELA B LOPEZ Y FLORENCI...,AVENIDA ANGELA B DE LOPEZ ESQUINA FLORENCIO SA...,INTERIOR NORDESTE,TACUAREMBO,CORDON,EL DORADO,EL DORADO,155,45


In [28]:
nielsen_final = pd.concat([nielsen, nielsen_nuevo], ignore_index=True).sort_values(by=['LOCL_COD',
                                                                                       'PRIORIDAD'])
nielsen_final

,STORE_ID,STORE,STORE_ADDRESS,AREA,SUB_AREA,CITY,RETAILER,BANNER,LOCL_COD,PRIORIDAD,GEOG_LOCL_ID
7,141175690,EL DORADO SUC 2021?18 DE JULIO 2006 ESQUINA PA...,18 DE JULIO 2006 ESQUINA-PABLO DE MARIA NN,GRAN MONTEVIDEO B,MONTEVIDEO,NaN,EL DORADO,EL DORADO,101,1.0,21
8,145749110,EL DORADO SUC 2022 PEATONAL SARANDI 584 MONTEV...,PEATONAL SARANDI 584 NN,GRAN MONTEVIDEO B,MONTEVIDEO,NaN,EL DORADO,EL DORADO,101,2.0,21
9,149665397,EL DORADO SUC 2023 18 DE JULIO 1807 MONTEVIDEO,18 DE JULIO 1807 NN,GRAN MONTEVIDEO B,MONTEVIDEO,NaN,EL DORADO,EL DORADO,101,3.0,21
10,164712852,EL DORADO SUC 2050 GARIBALDI 2579 MONTEVIDEO,GARIBALDI 2579 NN,GRAN MONTEVIDEO B,MONTEVIDEO,NaN,EL DORADO,EL DORADO,101,4.0,21
5,100573892,RED EXPRESS SUC 145 AVDA URUGUAY 1104 SECCION 106,AVDA URUGUAY 1104 NN,GRAN MONTEVIDEO B,MONTEVIDEO,MONTEVIDEO,TIENDA INGLESA,AUTOSERVICIOS TIENDA INGLESA,101,5.0,21
...,...,...,...,...,...,...,...,...,...,...,...
434,100356201,DEVOTO SUC. 3003 AV. GRAL RIVERA 4502 BRISAS G...,V GR L RIVER- 4502-BRIS S NN,GRAN MONTEVIDEO A,MONTEVIDEO,MALVIN,DEVOTO,DEVOTO,420,1.0,322
431,100356252,DEVOTO EXPRESS SUC. 3124 CNO. CARRASCO 6955 SE...,CNO C RR SCO 6955 NN,GRAN MONTEVIDEO A,MONTEVIDEO,NaN,DEVOTO,DEVOTO EXPRESS,420,2.0,322
430,100356186,DISCO SUC. 125 MCAL. SOLANO LOPEZ 1680 0,NN NN,GRAN MONTEVIDEO A,MONTEVIDEO,BUCEO,DISCO,DISCO,420,3.0,322
432,100356159,TIENDA INGLESA SUC. 1 AV. ITALIA ESQ. BOLIVIA ...,V IT LI--ESQ BOLIVI- 5820-M LVIN NN,GRAN MONTEVIDEO A,CARRASCO,CARRASCO,TIENDA INGLESA,TIENDA INGLESA,420,4.0,322


In [30]:
nielsen_final.to_excel('NIELSEN COMPLETO.xlsx', index=False)

In [10]:
f = datetime.now().replace(day=22, hour=19, minute=25, second=41).strftime('%Y%m%d_%H%M%S')

In [3]:
user, cursor, snow = snowflake_login()

Intento 1
Error: 251005: User is empty
Incorrect Password - provide again
Intento 2
Correct Password - connected to SNOWFLAKE


In [12]:
duplicados = descargar_segmento(cursor, 'DUPLICADOS', cond="'" + f + "'")
duplicados

,CODIGO_TIENDA,ORIN,PVP_NUEVO,EFFECTIVE_DATE,TABLA,FAMILIA,ARTC_ARTC_DESC,STCK_PRECIO_VENTA_DIA_CIVA


In [11]:
checks = descargar_segmento(cursor, query='TOTAL', cond="'" + f + "'")
checks

,ORIN,GEOG_LOCL_COD,PVP_NUEVO,EFFECTIVE_DATE,FAMILIA,INTEGRANTES_FAMILIA,CANASTA,ARTC_ARTC_DESC,ESTADO,ESTA_EN_PROMO,EXCLUIDO_COMERCIAL,PRECIO_DIFERENTE
0,1000505780,146,105,None,1000505777,3,surtido,GRISINES CRI CROC INTEGRALES EL TRIGAL 145 G,4,0,0,0
1,1000505779,168,105,None,1000505777,3,surtido,GRISINES CRI CROC CLASICOS EL TRIGAL 145 GR,4,0,0,0
2,1000504782,104,114,None,1000504782,1,surtido,GALLETA COFLER BA?ADAS CHOCOLATE 132.8 GR,4,1,0,1
3,1000057500,149,690,None,1000057500,1,surtido,QUESO SBRINZ PREMIUM FARMING COPETIN 1.00 KG,4,0,0,0
4,1000504782,329,114,None,1000504782,1,surtido,GALLETA COFLER BA?ADAS CHOCOLATE 132.8 GR,4,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
519,1000504782,168,114,None,1000504782,1,surtido,GALLETA COFLER BA?ADAS CHOCOLATE 132.8 GR,4,1,0,1
520,1000505777,145,105,None,1000505777,3,surtido,GRISINES EL TRIGAL CRI CROC SIN SAL 145 GR,4,0,0,0
521,1000157000,146,440,None,1000157000,1,referente,JAMON COCIDO DELIZIA SARUBBI.,4,0,0,0
522,1000504782,173,114,None,1000504782,1,surtido,GALLETA COFLER BA?ADAS CHOCOLATE 132.8 GR,4,1,0,1


In [14]:
final = descargar_segmento(cursor, query='FINAL', cond="'" + f + "'")
final

,ORIN,CODIGO_TIENDA,PVP_NUEVO
0,1000057447,151,680
1,1000057447,157,740
2,1000057301,159,450
3,1000353250,153,310
4,1000505780,159,105
...,...,...,...
390,1000057084,317,590
391,1000505780,318,105
392,1000505779,322,105
393,1000057156,323,550


In [15]:
datos = descargar_segmento(cursor, 'DATOS', cond="'" + f + "'")
datos

,ZONE_NODE_TYPE,ZONE_GROUP_ID,ZONE_ID,LOCATION,ITEM,DIFF_ID,SKULIST,LINK_CODE,CHANGE_TYPE,CHANGE_AMOUNT,...,PRICE_GUIDE_ID,EFFECTIVE_DATE,REASON_CODE,VENDOR_FUNDED_IND,FUNDING_TYPE,FUNDING_AMOUNT,FUNDING_AMOUNT_CURRENCY,FUNDING_PERCENT,DEAL_ID,DEAL_DETAIL_ID
0,0,None,None,151,1000057447,None,None,None,2,680,...,None,2025-07-23,2,0,None,None,None,None,None,None
1,0,None,None,157,1000057447,None,None,None,2,740,...,None,2025-07-23,2,0,None,None,None,None,None,None
2,0,None,None,159,1000057301,None,None,None,2,450,...,None,2025-07-23,2,0,None,None,None,None,None,None
3,0,None,None,153,1000353250,None,None,None,2,310,...,None,2025-07-23,2,0,None,None,None,None,None,None
4,0,None,None,159,1000505780,None,None,None,2,105,...,None,2025-07-23,2,0,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,0,None,None,317,1000057084,None,None,None,2,590,...,None,2025-07-23,2,0,None,None,None,None,None,None
391,0,None,None,318,1000505780,None,None,None,2,105,...,None,2025-07-23,2,0,None,None,None,None,None,None
392,0,None,None,322,1000505779,None,None,None,2,105,...,None,2025-07-23,2,0,None,None,None,None,None,None
393,0,None,None,323,1000057156,None,None,None,2,550,...,None,2025-07-23,2,0,None,None,None,None,None,None


In [7]:
with open('Queries/DUPLICADOS.sql', 'r', encoding="utf8") as file: command = file.read()
command

'WITH PP AS (\nSELECT\n    DISTINCT A.CODIGO_TIENDA, B.FAMILIA, COUNT(DISTINCT A.PVP_NUEVO) AS P\nFROM\n    SANDBOX_PLUS.DWH.INPUT_PRICING_ACUMULADO A\nLEFT JOIN\n    SANDBOX_PLUS.DWH.RESULTADO_PRICING_HISTORICO_BIS B\nON\n    A.ORIN = B.ORIN AND A.CODIGO_TIENDA = B.GEOG_LOCL_COD\nWHERE\n    FECHA_EJECUCION = (SELECT MAX(FECHA_EJECUCION) FROM SANDBOX_PLUS.DWH.RESULTADO_PRICING_HISTORICO_BIS)\nAND\n    TABLA={f}\nGROUP BY ALL\n)\n\nSELECT\n    A.*, B.FAMILIA, B.ARTC_ARTC_DESC, B.STCK_PRECIO_VENTA_DIA_CIVA\nFROM\n    SANDBOX_PLUS.DWH.INPUT_PRICING_ACUMULADO A\nJOIN\n    SANDBOX_PLUS.DWH.RESULTADO_PRICING_HISTORICO_BIS B\nON\n    A.ORIN = B.ORIN AND A.CODIGO_TIENDA = B.GEOG_LOCL_COD\nJOIN\n    PP\nON\n    A.CODIGO_TIENDA = PP.CODIGO_TIENDA AND B.FAMILIA = PP.FAMILIA AND P>1\nWHERE\n    FECHA_EJECUCION=(SELECT MAX(FECHA_EJECUCION) FROM SANDBOX_PLUS.DWH.RESULTADO_PRICING_HISTORICO_BIS)\nAND\n    TABLA={f}\nORDER BY\n    GEOG_LOCL_COD, B.FAMILIA;'

In [8]:
cursor.execute(command.format(f=f))

In [ ]:
d = cursor.fetch_pandas_all()
d

,CODIGO_TIENDA,ORIN,PVP_NUEVO,EFFECTIVE_DATE,TABLA,FAMILIA,ARTC_ARTC_DESC,STCK_PRECIO_VENTA_DIA_CIVA


In [5]:
c = "WHERE LGL.GEOG_LOCL_COD IN (101, 115) AND LAA.ORIN IN ('1000038127')"

In [12]:
with open('Queries/PRECIOS.sql', 'r', encoding="utf8") as file: command = file.read()
command

"WITH P AS (\nSELECT\n    ARTC_ARTC_ID, GEOG_LOCL_ID, MEDIAN(STCK_PRECIO_VENTA_DIA_CIVA) AS PRECIO_MEDIANO\nFROM\n    MSTRDB.DWH.FT_STOCK\nWHERE\n    TIEM_DIA_ID = CURRENT_DATE - 30\nGROUP BY ALL\n)\n\n, STOCK AS (\nSELECT\n    DISTINCT FS.ARTC_ARTC_ID, FS.GEOG_LOCL_ID, FS.STCK_PRECIO_VENTA_DIA_CIVA AS PVP_ACTUAL,\n    CASE\n        WHEN FCUR.ORIGIN_COUNTRY_ID = 'UYU' THEN FCUR.COSTO_UNITARIO_RMS\n        ELSE FS.STCK_COSTO_PROM_POND\n    END AS COSTO_UNITARIO,\n    COSTO_UNITARIO * ROUND(FS.STCK_PRECIO_VENTA_DIA_CIVA/FS.STCK_PRECIO_VENTA_DIA, 2) AS COSTO_UNITARIO_CON_IVA\nFROM\n    MSTRDB.DWH.FT_STOCK FS\nJOIN\n    MSTRDB.DWH.FT_COSTO_UNITARIO_RMS FCUR\nON\n    FS.GEOG_LOCL_ID = FCUR.GEOG_LOCL_ID AND FS.ARTC_ARTC_ID = FCUR.ARTC_ARTC_ID\n    AND FCUR.TIEM_DIA_ID = (SELECT MAX(TIEM_DIA_ID) FROM MSTRDB.DWH.FT_COSTO_UNITARIO_RMS)\nWHERE\n    FS.TIEM_DIA_ID = CURRENT_DATE - 1\n)\n\nSELECT\n    LAA.ORIN, LGL.GEOG_LOCL_COD AS LOCAL, PVP_ACTUAL, PRECIO_MEDIANO, COSTO_UNITARIO_CON_IVA,\n    RO

In [15]:
cursor.execute(command.format(cond=c))

In [16]:
p = cursor.fetch_pandas_all()
p

,ORIN,LOCAL,PVP_ACTUAL,PRECIO_MEDIANO,COSTO_UNITARIO_CON_IVA,MARGEN
0,1000038127,101,245.0,244.0,156.404,0.36
1,1000038127,115,198.0,244.0,156.404,0.21


In [4]:
duplicados = descargar_segmento(cursor, 'DUPLICADOS', cond=f)
duplicados

,CODIGO_TIENDA,ORIN,PVP_NUEVO,EFFECTIVE_DATE,TABLA,FAMILIA,ARTC_ARTC_DESC,STCK_PRECIO_VENTA_DIA_CIVA


In [7]:
p = descargar_segmento(cursor, 'PRECIOS', cond=c)
p

,ORIN,LOCAL,PVP_ACTUAL,PRECIO_MEDIANO,COSTO_UNITARIO_CON_IVA,MARGEN
0,1000038127,101,245.0,244.0,156.404,0.36
1,1000038127,115,198.0,244.0,156.404,0.21


In [ ]:
filtros_posibles = {'PROMOCION':'ESTA_EN_PROMO = 0',
                    'EXCLUIDOS COMERCIAL':'EXCLUIDO_COMERCIAL = 0',
                        'ACTIVOS':'ARTC_ESTA_ID = 4',
                        'PRECIO DIFERENTE':'S.STCK_PRECIO_VENTA_DIA_CIVA <> A.PVP_NUEVO'}
filtros = []
todos_los_filtros = []
for f in filtros:
    todos_los_filtros.append(filtros_posibles[f])
if len(todos_los_filtros) > 0:
    todos_los_filtros = ' AND ' + ' AND '.join(todos_los_filtros)
else:
    todos_los_filtros = ''
todos_los_filtros

''

In [18]:
' AND '.join(todos_los_filtros)

''

In [20]:
if len(todos_los_filtros) > 0:
    todos_los_filtros = ' AND ' + ' AND '.join(todos_los_filtros)